# Calculating vertical velocities
A guide to using the package XApRES to load existing xarrays and calculate vertical velocities

## Load the package and set up workspace

In [1]:
import sys
import sys
sys.path.append("../../xapres_package/")
import ApRESDefs
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr


In [2]:
xa = ApRESDefs.xapres(loglevel='debug')

DEBUG    15:36:26 	 ApRESDefs.py @function _setup_logging line 494 - Stream logging level set to DEBUG
DEBUG    15:36:26 	 ApRESDefs.py @function _setup_logging line 495 - Add console handler to logger
DEBUG    15:36:26 	 ApRESDefs.py @function _setup_logging line 508 - File logging level set to DEBUG


In [4]:
def reload(site):
    filename = f'gs://ldeo-glaciology/apres/greenland/2022/single_zarrs/{site}'
    ds = xr.open_dataset(filename,
        engine='zarr', 
        chunks={}) 
    return ds

ds_101 = reload("A101")
ds_103 = reload("A103")
ds_104 = reload("A104")

ValueError: unrecognized engine zarr must be one of: ['netcdf4', 'scipy', 'store']

In [4]:
filepaths = xa.list_files(directory='gs://ldeo-glaciology/GL_apres_2022', remote_load = True)
filepaths[0:5]

DEBUG    15:22:33 	 ApRESDefs.py @function list_files line 165 - Find all the dat files in the directory gs://ldeo-glaciology/GL_apres_2022 with remote_load = True
DEBUG    15:22:40 	 ApRESDefs.py @function list_files line 179 - Finish call to list_files. Found 386 files


['ldeo-glaciology/GL_apres_2022/A101/CardA/DIR2022-05-22-1939/DATA2022-05-22-1939.DAT',
 'ldeo-glaciology/GL_apres_2022/A101/CardA/DIR2022-05-26-1530/DATA2022-05-26-1530.DAT',
 'ldeo-glaciology/GL_apres_2022/A101/CardA/DIR2022-05-26-1536/DATA2022-05-26-1536.DAT',
 'ldeo-glaciology/GL_apres_2022/A101/CardA/DIR2022-05-26-1536/DATA2022-05-27-1506.DAT',
 'ldeo-glaciology/GL_apres_2022/A101/CardA/DIR2022-05-26-1536/DATA2022-05-28-1436.DAT']

## Load 3 Bursts

In [5]:
%%time 
import importlib
importlib.reload(ApRESDefs)  
xa = ApRESDefs.xapres(loglevel='debug', max_range=1400)
xa.load_all(directory='gs://ldeo-glaciology/GL_apres_2022/A101', 
            remote_load = True,
            file_numbers_to_process = [2,3,100], 
            bursts_to_process=[0,1]
           )
xa.data

DEBUG    15:23:08 	 ApRESDefs.py @function _setup_logging line 494 - Stream logging level set to DEBUG
DEBUG    15:23:08 	 ApRESDefs.py @function _setup_logging line 495 - Add console handler to logger
DEBUG    15:23:08 	 ApRESDefs.py @function _setup_logging line 508 - File logging level set to DEBUG
DEBUG    15:23:08 	 ApRESDefs.py @function list_files line 165 - Find all the dat files in the directory gs://ldeo-glaciology/GL_apres_2022/A101 with remote_load = True
DEBUG    15:23:08 	 ApRESDefs.py @function list_files line 179 - Finish call to list_files. Found 127 files
DEBUG    15:23:08 	 ApRESDefs.py @function load_all line 211 - Subset files to [2, 3, 100]
DEBUG    15:23:08 	 ApRESDefs.py @function load_all line 227 - Starting loop over dat files
DEBUG    15:23:08 	 ApRESDefs.py @function load_all line 230 - Load dat file ldeo-glaciology/GL_apres_2022/A101/CardA/DIR2022-05-26-1536/DATA2022-05-26-1536.DAT
DEBUG    15:23:16 	 ApRESDefs.py @function _all_bursts_in_dat_to_xarray line

DEBUG    15:23:37 	 ApRESDefs.py @function load_all line 240 - Concatenating all the multi-burst xarrays to create xapres.data
DEBUG    15:23:37 	 ApRESDefs.py @function _add_attrs line 409 - Adding attributes to the xapres.data
DEBUG    15:23:37 	 ApRESDefs.py @function load_all line 246 - Finish call to load_all. Call xapres.data to see the xarray this produced.
CPU times: user 9.2 s, sys: 3.19 s, total: 12.4 s
Wall time: 29.4 s


<xarray.Dataset>
Dimensions:          (time: 6, chirp_time: 40001, chirp_num: 20,
                      attenuator_setting_pair: 2, profile_range: 6658)
Coordinates:
  * time             (time) datetime64[ns] 2022-05-26T15:37:02 ... 2022-08-30...
  * chirp_time       (chirp_time) float64 0.0 2.5e-05 5e-05 ... 1.0 1.0 1.0
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
    filename         (time) <U83 'ldeo-glaciology/GL_apres_2022/A101/CardA/DI...
    burst_number     (time) int64 0 1 0 1 0 1
    AFGain           (attenuator_setting_pair) int64 -4 -14
    attenuator       (attenuator_setting_pair) float64 5.0 5.0
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 ...
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 ...
    latitude         (time) float64 68.71 68.71 68.71 68.71 68.71 68.71
    longitude        (time) float64 -49.55 -49.55 -49.55 -49.55 -49.55 -49.55
    battery_voltage  (time) float64 13.91 13.88 13.61 13.56 13.62 13.64
    temperature_1    (time) float64 3.008 4.773 8.5 7.32 2.812 4.969
    temperature_2    (time) float64 2.648 1.664 9.125 6.766 4.219 3.234

## Stack bursts

In [ ]:
# stack the data (this line stacks both the chirps and the profiles, but you could just do the profiles)
stacked1 = xa.data.isel(time=0, attenuator_setting_pair=0).mean(dim='chirp_num')
stacked2 = xa.data.isel(time=1, attenuator_setting_pair=0).mean(dim='chirp_num')
stacked3 = xa.data.isel(time=6, attenuator_setting_pair=0).mean(dim='chirp_num')


xa.dB(stacked1.profile).plot(label=stacked1.profile.time.data)
xa.dB(stacked2.profile).plot(label=stacked2.profile.time.data)
xa.dB(stacked3.profile).plot(label=stacked3.profile.time.data)
plt.legend()
plt.title('Range Profiles')


## Calculate vertical velocities for a pair of stacked bursts
The function looks like `generate_range_diff(self, data1, data2, win_cor, step, range_ext=None, win_wrap=10, thresh=0.9, uncertainty='noise_phasor')`. Changing the window for the coherence and the step creates pretty different results.

In [ ]:
for step in [10,20]:
    for win in [10,20,30]:
        vels, ds = xa.generate_range_diff(stacked1.profile,stacked2.profile,win,step)
        plt.plot(ds,vels,label=f'Coherence window: {win}, Coherence step: {step}')
plt.xlabel('depth [m]')
plt.ylabel('vertical velocity [m/s]')
plt.legend()

With times split further apart, this windowing and step size has even more impact. 

In [ ]:
for step in [10,20]:
    for win in [10,20,30]:
        vels, ds = xa.generate_range_diff(stacked1.profile,stacked3.profile,win,step)
        plt.plot(ds,vels,label=f'Coherence window: {win}, Coherence step: {step}')
plt.xlabel('depth [m]')
plt.ylabel('vertical velocity [m/s]')
plt.legend()

## Testing with multiple bursts in data

Given the example below, clearly at the moment something causes the range_diff calculation to collapse, perhaps it doesn't like working with dimensions. A possible solution is to have a for loop iterating through each pair to compare. This is simple to implement, but could be annoying with big data (though I see that in generating the xarrays, there's for loops anyways). Another solution that I need to sleep on is to modify the calculations form ImpDAR to accomodate the xarray structure (since currently it is only operating with numpy arrays). This may be a bit trickier as in I can't think of the immediate implementation. 

In [ ]:
stacked_arr1 = xa.data.isel(time=[0,1,2,3], attenuator_setting_pair=0).mean(dim='chirp_num')
stacked_arr2 = xa.data.isel(time=[1,2,3,4], attenuator_setting_pair=0).mean(dim='chirp_num')

In [ ]:
vels, ds = xa.generate_range_diff(stacked_arr1.profile,stacked_arr2.profile,win,step)

In [ ]:
vels

In [ ]:
ds